In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [20]:
#coding:utf-8
import numpy as np
import json
# from model import ConceptFlow, use_cuda
# from preprocession import prepare_data, build_vocab, gen_batched_data
import torch
import warnings
import yaml
import os
import requests
import re
import gensim.downloader as api
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import utils as nn_utils
warnings.filterwarnings('ignore')

In [21]:
w2v_model = api.load('word2vec-google-news-300')

In [3]:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    device = torch.device('cuda:0')

In [4]:
%run '/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/Central.ipynb'
%run '/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/Embedding.ipynb'
%run '/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/Outer.ipynb'
%run '/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/Conceptflow.ipynb'
%run '/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/Preprocession.ipynb'

Exception: File `"'/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592.py"` not found.

In [2]:
data_dir = "D:\\13.PURDUE\\Data\\ConceptFlow(ECCF)_data"
with open('%s/resource.txt' % data_dir) as f:
    d = json.loads(f.readline())

    csk_triples = d['csk_triples']
    csk_entities = d['csk_entities']
    raw_vocab = d['vocab_dict']
    kb_dict = d['dict_csk']
    dict_csk_entities = d['dict_csk_entities']
    dict_csk_triples = d['dict_csk_triples']

# New post preprocessing

In [214]:
question = ''
res = re.findall( r'\w+|[^\s\w]+', questions)

def query_conceptnet(keyword):
    url = f"http://api.conceptnet.io/c/en/{keyword}?limit=100"
    response = requests.get(url)
    return response.json()

def extract_data(concept_data):
    entities = set()
    triples = []
    for edge in concept_data['edges']:
        start = edge['start']['@id'].split('/')[-1]
        end = edge['end']['@id'].split('/')[-1]
        rel = edge['rel']['@id'].split('/')[-1]
        triples.append((start, rel, end))
        entities.update([start, end])
    return entities, triples


# if res not in csk_entities:
#     print('Come again? Could you ask again a little bit detail?')

    
def generate_response_format(dict_csk_entities, dict_csk_triples, post, entities, triples):
    res = re.findall( r'\w+|[^\s\w]+', post)

    # Simulate entity and triple index mapping
    entity2id = {entity: idx for idx, entity in enumerate(entities)}
    triple_indices = 0  # Assuming each triple gets a unique index
    post_triples = np.zeros(len(res))
    
#     for idx, split in enumerate(res):
#         if split in dict_csk_entities:
            
        
# #         else:
# #             continue
        
            
    
    data_format = {
        "all_entities_one_hop": list(entity2id.values()),
        "post_triples": list(triple_indices),  # Example: every triple is used
        "post": res,
        "response": ["This", "is", "an", "example", "response", "."],
        "match_response_index_one_hop": [-1] * len(entity2id),  # Simplified example
        "only_two": list(entity2id.values())[::2],  # Every second entity for example
        "match_response_index_only_two": [-1] * len(entity2id),  # Simplified example
        "all_triples_one_hop": list(triple_indices)  # Simplified example
    }
    return data_format

def main(question):
    keywords = question.split()  # Simple keyword extraction from the question
    all_entities = set()
    all_triples = []

    for keyword in keywords:
        concept_data = query_conceptnet(keyword)
#         print('concept_data',concept_data)
        entities, triples = extract_data(concept_data)
        print('entities', entities)
        print('triples', triples)
        all_entities.update(entities)
        all_triples.extend(triples)

    formatted_data = generate_response_format(all_entities, all_triples)
    print(formatted_data)

# Example usage
main("What is related to coffee?")

entities set()
triples []
entities {'ειναι', 'ما_هذا', 'ドス', 'wast', 'has', 'oes', 'bes', 'е', 'でげす', 'inar', 'ざり', 'です', 'már_késő_van', "where's", 'isness', 'зарем', 'いらっしゃる', 'je', 'ada_apa', 'mura', 'er', 'e', 'cérb', 'nĩ', 'es', 'found_in_subway', 'αληθώσ_ανέστη', 'id', 'sta', 'qu_est_ce_que_c_est', 'っす', "she'sn't", "who's", 'يكون', 'だ', 'で御座ります', 'どす', 'is', 'expensive', 'ei_ole', 'v', 'é', 'trínarb', 'ialah', 'está', 'a', 'en_1', 'bilgisayarsa', '3', '𐌹𐍃𐍄', 'で御座います', 'beest', 'r', 'infinite_stratos', 'n', 'tabansızsa', '𐬀𐬯𐬙𐬌', 'じゃ', 'ist', 'ってのは', 'єстъ', '是', 'є', 'ónar', 'evse', 'thì', 'isentropic', 'being', 'jeste', 'と言うのは', 'adalah', 'átʼéii', 'yw', 'IS_(Infinite_Stratos)', 'your+name+is+mud-p#Component-3', 'jest', 'でごんす', 'beeth', 'これはなんですか', 'dar', 'computing', "it's"}
triples [('どす', 'Synonym', 'is'), ('n', 'Synonym', 'is'), ('en_1', 'RelatedTo', 'is'), ('ざり', 'Synonym', 'is'), ('is', 'Synonym', 'jest'), ('ドス', 'Synonym', 'is'), ('いらっしゃる', 'Synonym', 'is'), ('is', 'Synon

TypeError: generate_response_format() missing 3 required positional arguments: 'post', 'entities', and 'triples'

In [3]:
post = "I wear glasses and at the age of 17 I would have been mortified to be seen in a photo wearing them."
post2 = "As the comet streaked across the Chicago sky, I felt as though time had looped back to the Wild West, where cowboys would shoot at such celestial wonders with their six-shooters."
post3 = "this fucking shit pisses me off to no end, when these fucking liberal hypocrites imply the only group of people capable of being racist are the whites."

# create_all_entities_one_hop & create_all_triples_one_hop

In [79]:
csk_triples = csk_triples
dict_csk_entities = dict_csk_entities
csk_entities = csk_entities
res = re.findall( r'\w+|[^\s\w]+', post3)
post_entities = []

tmp_e = ''
tmp_o = ''

for i in res:
    if i in csk_entities:
        post_entities.append(i)
    else:
        continue
print('post_entities',post_entities)


# find one_hop_triples from post entities
all_triples_one_hop = []

# find one_hop_entities from post entities
tmp_id_one_hop = []

threshold = 0.24
for e_o in csk_triples:
    e_o_split = e_o.split(',')
    for entity in post_entities:
        tmp_e = e_o_split[0]
        tmp_o = e_o_split[2].lstrip()
        tmp_pair = [tmp_e, tmp_o]
        if entity in tmp_pair:
            if entity == tmp_pair[0] and tmp_pair[1] not in post_entities:
                if entity in w2v_model.key_to_index and tmp_pair[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score = w2v_model.similarity(entity,tmp_pair[1])
                    all_triples_one_hop.append(dict_csk_triples[e_o])
                    if tmp_pair[1] in dict_csk_entities and sim_score>threshold:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])                    
                else:
                    all_triples_one_hop.append(dict_csk_triples[e_o])
                    if tmp_pair[1] in dict_csk_entities:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[1]])
                        
            elif entity == tmp_pair[1] and tmp_pair[0] not in post_entities:
                if entity in w2v_model.key_to_index and tmp_pair[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score = w2v_model.similarity(entity,tmp_pair[0])
                    if tmp_pair[0] in dict_csk_entities and sim_score>threshold:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])
                        all_triples_one_hop.append(dict_csk_triples[e_o])
                else:
                    if tmp_pair[0] in dict_csk_entities:
                        all_triples_one_hop.append(dict_csk_triples[e_o])
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair[0]])

            else:
                continue
        else:
            continue
        tmp_e = []
        tmp_o = []
        tmp_pair = []

id_one_hop = [i for n, i in enumerate(tmp_id_one_hop) if i not in tmp_id_one_hop[:n]]

# print(id_one_hop)
print(len(id_one_hop))
print(id_one_hop)
print(len(all_triples_one_hop))

post_entities ['fucking', 'shit', 'end', 'fucking', 'liberal', 'hypocrites', 'imply', 'group', 'capable', 'racist', 'whites']
125
[16196, 20496, 6055, 7523, 12453, 17645, 10558, 7770, 18128, 20994, 7740, 11761, 21894, 4786, 1185, 12549, 13720, 13719, 6491, 7572, 7362, 10411, 12220, 11105, 18812, 6678, 7887, 938, 15041, 5825, 4154, 12978, 12237, 19142, 9075, 2826, 21601, 21737, 1545, 16853, 21892, 14247, 3736, 12652, 1637, 21217, 14599, 2541, 20911, 9618, 4802, 13804, 5399, 20353, 19107, 15904, 19685, 6122, 942, 3571, 1606, 3487, 17142, 8545, 5192, 19351, 1974, 18311, 1802, 5562, 11938, 14895, 12380, 10576, 8695, 20626, 10083, 17116, 21079, 10252, 11450, 362, 5173, 9548, 2024, 4640, 6821, 21852, 19031, 659, 8360, 18713, 2846, 15433, 2889, 8841, 3078, 14950, 9310, 16988, 3164, 3159, 15063, 14042, 18869, 2002, 8488, 13565, 12866, 12811, 10057, 4518, 16111, 6384, 17607, 9474, 12896, 7829, 7766, 14614, 12509, 5800, 12441, 16320, 20026]
297


# Only_two

In [81]:
tmp_e2 = ''
tmp_o2 = ''
tmp_pair2= []
# find two hope from post entities
lsk = ['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat']
# id2word = dict()
# for key in word2id.keys():
#     id2word[word2id[key]] = key
swapped_dict = {value: key for key, value in dict_csk_entities.items()}
entities_one_hop = []
for i in id_one_hop:
    entities_one_hop.append(swapped_dict[i])
entities_one_hop2 = entities_one_hop[:5]
threshold2 = 0.01
tmp_entities_two_hop = []
for e_o2 in csk_triples:
    e_o2 = e_o2.split(',')
    for entity2 in entities_one_hop:
        tmp_e2 = e_o2[0]
        tmp_o2 = e_o2[2].lstrip()
        tmp_pair2 = [tmp_e2, tmp_o2]
        if entity2 in tmp_pair2:
            if entity2 == tmp_pair2[0] and tmp_pair2[1] not in post_entities and tmp_pair2[1] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[1])
                    if tmp_pair2[1] in dict_csk_entities and sim_score2 > threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[1]])                  
                    else:
                        if tmp_pair2[1] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[1]])
            elif entity2 == tmp_pair2[1] and tmp_pair2[0] not in post_entities and tmp_pair2[0] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[0])
                    if tmp_pair2[0] in dict_csk_entities and sim_score2>threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[0]])                  
                    else: 
                        if tmp_pair2[0] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[0]])
            else:
                continue
        else:
            continue
        tmp_e2 = []
        tmp_o2 = []
        tmp_pair2 = []

# print(len(tmp_entities_two_hop))
id_two_hop = [i for n, i in enumerate(tmp_entities_two_hop) if i not in tmp_entities_two_hop[:n]]
print(len(id_two_hop))
print(id_two_hop)
# print(entities_two_hop)

176
[109, 18698, 1955, 17681, 17989, 589, 17252, 2231, 18661, 6617, 2581, 3097, 8527, 9411, 15859, 9619, 7784, 18870, 17092, 10552, 996, 1499, 14741, 9244, 19574, 15724, 17692, 849, 21573, 20739, 696, 10376, 5272, 1039, 12364, 16268, 20748, 20987, 16744, 16781, 16740, 14295, 17836, 16042, 7154, 20896, 5031, 10653, 13169, 4075, 12181, 7432, 8050, 15146, 20213, 5694, 720, 4272, 12060, 20754, 13236, 14496, 20717, 15662, 3983, 6654, 4606, 9448, 6881, 19949, 875, 13343, 21012, 12391, 20534, 334, 7738, 4889, 3041, 282, 18823, 11880, 11601, 14723, 6391, 15805, 9718, 15799, 2629, 17685, 6968, 6676, 14582, 15744, 18399, 13163, 17442, 14297, 943, 19511, 13866, 1483, 914, 9810, 21830, 12534, 13364, 7746, 2653, 13654, 13124, 10276, 1737, 15732, 1572, 7452, 9138, 19325, 21903, 17046, 536, 17723, 13903, 13927, 19294, 20039, 8947, 20237, 1771, 3047, 521, 8090, 17108, 1988, 20528, 12647, 9010, 18099, 6531, 17633, 18960, 17023, 17746, 5428, 14674, 18405, 21358, 13277, 3305, 8856, 22518, 8616, 1774, 514

In [82]:
print(np.sort(id_one_hop))
print(np.sort(id_two_hop))

[  362   659   938   942  1185  1545  1606  1637  1802  1974  2002  2024
  2541  2826  2846  2889  3078  3159  3164  3487  3571  3736  4154  4518
  4640  4786  4802  5173  5192  5399  5562  5800  5825  6055  6122  6384
  6491  6678  6821  7362  7523  7572  7740  7766  7770  7829  7887  8360
  8488  8545  8695  8841  9075  9310  9474  9548  9618 10057 10083 10252
 10411 10558 10576 11105 11450 11761 11938 12220 12237 12380 12441 12453
 12509 12549 12652 12811 12866 12896 12978 13565 13719 13720 13804 14042
 14247 14599 14614 14895 14950 15041 15063 15433 15904 16111 16196 16320
 16853 16988 17116 17142 17607 17645 18128 18311 18713 18812 18869 19031
 19107 19142 19351 19685 20026 20353 20496 20626 20911 20994 21079 21217
 21601 21737 21852 21892 21894]
[  109   269   276   282   334   521   536   589   696   720   849   875
   914   943   996  1039  1483  1499  1572  1722  1737  1771  1774  1955
  1988  2231  2581  2629  2653  2717  3041  3047  3097  3209  3305  3757
  3983  4075  4272 

In [98]:
same = [i for i, j in zip(id_one_hop, id_two_hop) if i == j]
print(same)

[]


# create_post_triples

In [35]:
# Function to create the post_triples list
def create_post_triples(post, csk_entities):
    csk_entities = csk_entities
    post_csk_entities = {}
    count = 1
    res = re.findall( r'\w+|[^\s\w]+', post)
    print(res)
    print(len(res))
    for i in res:
#         print(i)
        if i in post_csk_entities.keys():
            continue

        elif i in csk_entities:
            post_csk_entities[i] = count
            count = count+1
        else:
            continue
            
    post_triples = []
    for word in res:
        if word in post_csk_entities:
            post_triples.append(post_csk_entities[word])
        else:
            post_triples.append(0)  # Use 0 for words that are not in dict_csk_entities
    return post_triples

In [38]:
post_triples = create_post_triples(post3, csk_entities)

print("Post Triples:", post_triples)

['this', 'fucking', 'shit', 'pisses', 'me', 'off', 'to', 'no', 'end', ',', 'when', 'these', 'fucking', 'liberal', 'hypocrites', 'imply', 'the', 'only', 'group', 'of', 'people', 'capable', 'of', 'being', 'racist', 'are', 'the', 'whites', '.']
29
Post Triples: [0, 1, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 4, 5, 6, 0, 0, 7, 0, 0, 8, 0, 0, 9, 0, 0, 10, 0]


# one_two_triple

In [61]:
ack = csk_triples[:10]
print(ack)

['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat', 'muscular, RelatedTo, strength', 'glass, RelatedTo, ceramic', 'shad, RelatedTo, cool', 'remind, RelatedTo, memory', 'honorable, Synonym, mention']


In [73]:
t = 1
b = 'a'
if ['a', 1] == [t,b]:
    print('true1')
elif ['a',1] == [b,t]:
    print('true2')

true2


In [117]:
tmp_e2 = ''
tmp_o2 = ''
tmp_pair2= []
# find two hope from post entities
lsk = ['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat']
# id2word = dict()
# for key in word2id.keys():
#     id2word[word2id[key]] = key
swapped_dict = {value: key for key, value in dict_csk_entities.items()}
entities_one_hop = []
for i in id_one_hop:
    entities_one_hop.append(swapped_dict[i])
threshold2 = 0.01
tmp_entities_two_hop = []

# ☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆
one_two_triples = []
tmp_one_two_triples = []
# tmp_one_two_triples.append(dict_csk_triples[e_o])
# one_two_triples.append(tmp_one_two_triples)
for e_o2 in csk_triples:
    e_o2_split = e_o2.split(',')
    for entity2 in entities_one_hop:
        tmp_e2 = e_o2_split[0]
        tmp_o2 = e_o2_split[2].lstrip()
        tmp_pair2 = [tmp_e2, tmp_o2]
        if entity2 in tmp_pair2:
            if entity2 == tmp_pair2[0] and tmp_pair2[1] not in post_entities and tmp_pair2[1] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[1] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[1])
                    if tmp_pair2[1] in dict_csk_entities and sim_score2 > threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[1]])
                    else:
                        if tmp_pair2[1] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[1]])
            elif entity2 == tmp_pair2[1] and tmp_pair2[0] not in post_entities and tmp_pair2[0] not in entities_one_hop:
                if entity2 in w2v_model.key_to_index and tmp_pair2[0] in w2v_model.key_to_index: # check both of them in w2vec vocab
                    sim_score2 = w2v_model.similarity(entity2,tmp_pair2[0])
                    if tmp_pair2[0] in dict_csk_entities and sim_score2>threshold2:
                        tmp_id_one_hop.append(dict_csk_entities[tmp_pair2[0]])
                    else: 
                        if tmp_pair2[0] in dict_csk_entities:
                            tmp_entities_two_hop.append(dict_csk_entities[tmp_pair2[0]])
            else:
                continue
        else:
            continue
        tmp_e2 = []
        tmp_o2 = []
        tmp_pair2 = []
# print(len(tmp_entities_two_hop))
id_two_hop = [i for n, i in enumerate(tmp_entities_two_hop) if i not in tmp_entities_two_hop[:n]]
print(len(id_two_hop))
print(id_two_hop)
# print(entities_two_hop)


176
[109, 18698, 1955, 17681, 17989, 589, 17252, 2231, 18661, 6617, 2581, 3097, 8527, 9411, 15859, 9619, 7784, 18870, 17092, 10552, 996, 1499, 14741, 9244, 19574, 15724, 17692, 849, 21573, 20739, 696, 10376, 5272, 1039, 12364, 16268, 20748, 20987, 16744, 16781, 16740, 14295, 17836, 16042, 7154, 20896, 5031, 10653, 13169, 4075, 12181, 7432, 8050, 15146, 20213, 5694, 720, 4272, 12060, 20754, 13236, 14496, 20717, 15662, 3983, 6654, 4606, 9448, 6881, 19949, 875, 13343, 21012, 12391, 20534, 334, 7738, 4889, 3041, 282, 18823, 11880, 11601, 14723, 6391, 15805, 9718, 15799, 2629, 17685, 6968, 6676, 14582, 15744, 18399, 13163, 17442, 14297, 943, 19511, 13866, 1483, 914, 9810, 21830, 12534, 13364, 7746, 2653, 13654, 13124, 10276, 1737, 15732, 1572, 7452, 9138, 19325, 21903, 17046, 536, 17723, 13903, 13927, 19294, 20039, 8947, 20237, 1771, 3047, 521, 8090, 17108, 1988, 20528, 12647, 9010, 18099, 6531, 17633, 18960, 17023, 17746, 5428, 14674, 18405, 21358, 13277, 3305, 8856, 22518, 8616, 1774, 514

In [142]:
one_two_triples = []
tmp_one_two_triples = []
count = 0
for o_h_t_i in id_one_hop:
    o_h_e = swapped_dict[o_h_t_i]
    tmp_one_two_triples = []
    for t_h_t_i in id_two_hop:
        t_h_e = swapped_dict[t_h_t_i]
        for e_o3 in csk_triples:
            e_o3_split = e_o3.split(',')
            tmp_e3 = e_o3_split[0]
            tmp_o3 = e_o3_split[2].lstrip()
            if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]:
                print('yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]')
                tmp_one_two_triples.append(dict_csk_triples[e_o3])
            elif [o_h_e, t_h_e] == [tmp_o3, tmp_e3]:
                print('yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]')
                tmp_one_two_triples.append(dict_csk_triples[e_o3])
            else:
                continue
    print(tmp_one_two_triples)
    one_two_triples.append(tmp_one_two_triples)
    if count // 10 == 0:
        print('10th o_h_t_i')
            
print('len_one_two_triples',len(one_two_triples))
print(one_two_triples)

[]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
[42515, 94866]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
[76846]
10th o_h_t_i
[]
10th o_h_t_i
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
[94514, 79026, 43932]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o_h_t_i
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
yes2 if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]
[1032, 31295, 41884, 87201, 48031, 84792, 94719, 63152, 89554]
10th o_h_t_i
[]
10th o_h_t_i
[]
10th o

KeyboardInterrupt: 

In [124]:
# [lmfao-16196, 20496, 6055, 7523, 12453]
# [adapt-109, 18698, 1955, 17681, 17989]
d_csk_triples = {"lmfao, RelatedTo, adapt": 0, "lmfao, RelatedTo, gorge":1, "lmfao, RelatedTo, assembly":2, "goon, RelatedTo, malt":3, "frigging, RelatedTo, boat":4}


In [134]:
id_one_hop_ex = id_one_hop[:5]
id_two_hop_ex = id_two_hop[:5]
cst = csk_triples[:5]
ccc_triples = ['lmfao, RelatedTo, adapt', 'gorge, RelatedTo, lmfao', "lmfao, RelatedTo, assembly", "goon, RelatedTo, malt", "torture, RelatedTo, finish"]
d_csk_triples = {"lmfao, RelatedTo, adapt": 0, "gorge, RelatedTo, lmfao":1, "lmfao, RelatedTo, assembly":2, "goon, RelatedTo, malt":3, "torture, RelatedTo, finish":4}

# 1955 -assembly
print(id_one_hop_ex)
print(id_two_hop_ex)
print(cst)

[16196, 20496, 6055, 7523, 12453]
[109, 18698, 1955, 17681, 17989]
['loop, RelatedTo, chicago', 'shoot, RelatedTo, comet', 'dumb, RelatedTo, deaf', 'walk, RelatedTo, door', 'pier, RelatedTo, boat']


In [136]:
# example
one_two_triples = []
count = 0
for o_h_t_i in id_one_hop_ex:
    o_h_e = swapped_dict[o_h_t_i]
    tmp_one_two_triples = []
    for t_h_t_i in id_two_hop_ex:
        t_h_e = swapped_dict[t_h_t_i]
        for e_o3 in ccc_triples:
            e_o3_split = e_o3.split(',')
            tmp_e3 = e_o3_split[0]
            tmp_o3 = e_o3_split[2].lstrip()
#             print('o_h_e:', o_h_e)
#             print('t_h_e', t_h_e)
            if [o_h_e, t_h_e] == [tmp_e3, tmp_o3]:
                tmp_one_two_triples.append(d_csk_triples[e_o3])
            elif [o_h_e, t_h_e] == [tmp_o3, tmp_e3]:
                tmp_one_two_triples.append(d_csk_triples[e_o3])                
            else:
                continue
    one_two_triples.append(tmp_one_two_triples)
 
            
print('len_one_two_triples',len(one_two_triples))
print(one_two_triples)

len_one_two_triples 5
[[0, 1, 2], [3], [], [4], []]


# Utils

In [6]:
def padding(sent, l):
    return sent + ['_EOS'] + ['_PAD'] * (l-len(sent)-1)

def padding_triple_id(entity2id, triple, num, l):
    newtriple = []
    for i in range(len(triple)):
        for j in range(len(triple[i])):
            for k in range(len(triple[i][j])):
                if triple[i][j][k] in entity2id:
                    triple[i][j][k] = entity2id[triple[i][j][k]]
                else:
                    triple[i][j][k] = entity2id['_NONE']

    for tri in triple:
        newtriple.append(tri + [[entity2id['_PAD_H'], entity2id['_PAD_R'], entity2id['_PAD_T']]] * (l - len(tri)))
    pad_triple = [[entity2id['_PAD_H'], entity2id['_PAD_R'], entity2id['_PAD_T']]] * l
    return newtriple + [pad_triple] * (num - len(newtriple))

def build_kb_adj_mat(kb_adj_mats, fact_dropout):
    """Create sparse matrix representation for batched data"""
    mats0_batch = np.array([], dtype=int)
    mats0_0 = np.array([], dtype=int)
    mats0_1 = np.array([], dtype=int)
    vals0 = np.array([], dtype=float)

    mats1_batch = np.array([], dtype=int)
    mats1_0 = np.array([], dtype=int)
    mats1_1 = np.array([], dtype=int)
    vals1 = np.array([], dtype=float)

    for i in range(kb_adj_mats.shape[0]):
        (mat0_0, mat0_1, val0), (mat1_0, mat1_1, val1) = kb_adj_mats[i]
        assert len(val0) == len(val1)
        num_fact = len(val0)
        num_keep_fact = int(np.floor(num_fact * (1 - fact_dropout)))
        mask_index = np.random.permutation(num_fact)[ : num_keep_fact]
        # mat0
        mats0_batch = np.append(mats0_batch, np.full(len(mask_index), i, dtype=int))
        mats0_0 = np.append(mats0_0, mat0_0[mask_index])
        mats0_1 = np.append(mats0_1, mat0_1[mask_index])
        vals0 = np.append(vals0, val0[mask_index])
        # mat1
        mats1_batch = np.append(mats1_batch, np.full(len(mask_index), i, dtype=int))
        mats1_0 = np.append(mats1_0, mat1_0[mask_index])
        mats1_1 = np.append(mats1_1, mat1_1[mask_index])
        vals1 = np.append(vals1, val1[mask_index])

    return (mats0_batch, mats0_0, mats0_1, vals0), (mats1_batch, mats1_0, mats1_1, vals1)

# Generating

In [7]:
csk_triples, csk_entities, kb_dict = [], [], []
dict_csk_entities, dict_csk_triples = {}, {}
class Config():
  def __init__(self, path):
    self.config_path = path
    self._get_config()

  def _get_config(self):
    with open(self.config_path, "r") as setting:
      config = yaml.load(setting,Loader=yaml.FullLoader)
    self.is_train = config['is_train']
    self.test_model_path = config['test_model_path']
    self.embed_units = config['embed_units']
    self.symbols = config['symbols']
    self.units = config['units']
    self.layers = config['layers']
    self.batch_size = config['batch_size']
    self.data_dir = config['data_dir']
    self.num_epoch = config['num_epoch']
    self.lr_rate = config['lr_rate']
    self.lstm_dropout = config['lstm_dropout']
    self.linear_dropout = config['linear_dropout']
    self.max_gradient_norm = config['max_gradient_norm']
    self.trans_units = config['trans_units']
    self.gnn_layers = config['gnn_layers']
    self.fact_dropout = config['fact_dropout']
    self.fact_scale = config['fact_scale']
    self.pagerank_lambda = config['pagerank_lambda']
    self.result_dir_name = config['result_dir_name']
    self.generated_path = config['generated_path']

  def list_all_member(self):
    for name, value in vars(self).items():
        print('%s = %s' % (name, value))


def run(model, data_train, config, word2id, entity2id):
    batched_data = gen_batched_data(data_train, config, word2id, entity2id)
    if model.is_inference == True:
        word_index, selector = model(batched_data)
        return word_index, selector
    else:
        decoder_loss, sentence_ppx, sentence_ppx_word, sentence_ppx_local, sentence_ppx_only_two, word_neg_num, local_neg_num, only_two_neg_num = model(batched_data)
        return decoder_loss, sentence_ppx, sentence_ppx_word, sentence_ppx_local, sentence_ppx_only_two, word_neg_num, local_neg_num, only_two_neg_num


def generate(model, data_test, config, word2id, entity2id, epoch = 0, model_path = None):
  if model_path != None:
      model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

  count = 0
  model.is_inference = True
  id2word = dict()
  for key in word2id.keys():
      id2word[word2id[key]] = key

  def write_batch_res_text(word_index, id2word, selector = None):
      w = open(config.generated_path + '/generated_res_Scr.txt', 'a')
      batch_size = len(word_index)
      decoder_len = len(word_index[0])
      text = []
      if selector != None:
          for i in range(batch_size):
              tmp_dict = dict()
              tmp = []
              for j in range(decoder_len):
                  if word_index[i][j] == 2:
                      break
                  tmp.append(id2word[word_index[i][j]])
              # print(tmp)
              tmp_dict['res_text'] = tmp
              local_tmp = []
              only_two_tmp = []
              for j in range(len(tmp)):
                  if selector[i][j] == 1:
                      local_tmp.append(tmp[j])
                  if selector[i][j] == 2:
                      only_two_tmp.append(tmp[j])
              tmp_dict['local'] = local_tmp
              tmp_dict['only_two'] = only_two_tmp
              text.append(tmp_dict)

      w.write(json.dumps(model_path+ '\n'))
      for line in text:
          print(line)
          w.write(json.dumps(line) + '\n')
      w.close()

  for iteration in range(len(data_test)):
    word_index, selector = run(model, data_test[(iteration * config.batch_size):(iteration * \
          config.batch_size + config.batch_size)], config, word2id, entity2id)

    if count % 50 == 0:
        print ("generate:", iteration)
    count += 1
    write_batch_res_text(word_index, id2word, selector=selector)


def main():
  config = Config('/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/config.yml')
  config.list_all_member()
  raw_vocab, _, data_test = prepare_data(config)
  word2id, entity2id, vocab, embed, entity_vocab, entity_embed, relation_vocab, relation_embed, entity_relation_embed = build_vocab(config.data_dir, raw_vocab, config = config)
  model = use_cuda(ConceptFlow(config, embed, entity_relation_embed))

  model_optimizer = torch.optim.Adam(model.parameters(), lr = config.lr_rate)

  if not os.path.exists(config.generated_path):
      os.mkdir(config.generated_path)

  generate(model, data_test, config, word2id, entity2id, model_path=config.test_model_path)

In [27]:
os.path.realpath('/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/config.yml')

'/content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/config.yml'

In [8]:
main()

config_path = /content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/config.yml
is_train = False
test_model_path = /content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/training_output/_epoch_4.pkl
embed_units = 300
symbols = 30000
units = 512
layers = 2
batch_size = 30
data_dir = /content/drive/MyDrive/Colab Notebooks/ConceptFlow(ECCF)_data
num_epoch = 5
lr_rate = 0.0001
lstm_dropout = 0.3
linear_dropout = 0.2
max_gradient_norm = 5
trans_units = 100
gnn_layers = 3
fact_dropout = 0.0
fact_scale = 1
pagerank_lambda = 0.8
result_dir_name = /content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/training_output
generated_path = /content/drive/MyDrive/Commit_colab/PURDUE-2024-SPRING/CS592 HAI/Project/inference_output
Creating word vocabulary...
Creating entity vocabulary...
Creating relation vocabulary...
Loading word vectors...
    processing line 0
    processing line 100000
    processing line 200000
    processing line 300000
    pr

ValueError: max() arg is an empty sequence